# **Acquiring and Processing Information**
In this project, I will work with Olymlic Tokyo dataset and perform the operations of Extraction, Transformation, and Loading (ETL) as required.

- Task 1: Logging function
- Task 2: Extraction of data
- Task 3: Transformation of data
- Task 4: Loading to CSV
- Task 5: Loading to Database
- Task 6: Function to Run queries on Database
- Task 7: Verify log entries

### **Preliminaries: Setting environment, Installing libraries and downloading data**
Setting evironment:
- Set up a virtual environment for Python: 
   - Run command in terminal: *"python -m venv env"*
- Activate the env folder:
   - Run command in terminal: *".\env\Scripts\activate"*

- **NOTE:** To deactivate env folder:
   - Run command in terminal: *"deactivate"*

Install the required libraries
- All libraries for this project in requirements.txt file.
  - Run command in terminal: *"pip install -r .\requirements.txt"*

Downloading data
- I use dataset on Kaggel.com: [2021 Olympics in Tokyo](https://www.kaggle.com/datasets/arjunprasadsarkhel/2021-olympics-in-tokyo)
- Data folder contains 2 subfolder (data/raw and data/processed)

### **Dataset Introduction**
This contains the details of over 11,000 athletes, with 47 disciplines, along with 743 Teams taking part in the 2021(2020) Tokyo Olympics. This dataset contains the details of the Athletes, Coaches, Teams participating as well as the Entries by gender. It contains their names, countries represented, discipline, gender of competitors, name of the coaches.

- Athletes.xlsx : Contains details about the participating Athletes( Name (name of the athlete) , NOC (Country) , Discipline )
- Coaches.xlsx : Contains details about the Coach(Country, Discipline, Event)
- EntriesGender.xlsx : Contains details about the Coach(Country, Discipline, Event)
- Medals.xlsx : Medals as on 29th July 2021
- Teams.xlsx : Contains the details of all the Teams(Country, event, Discipline, Event)

**NOTE:** All data raw file in data/raw folder.

### **Importing libraries**
Importing the required libraries

In [67]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pyodbc

# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

### **Task**
#### **Task 1: Logging function**
- This function accepts the message to be logged and enters it to the log file on a new line.
- Log entries will be in the format: ```<timestamp> : <message>```

In [68]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''

    timestamp_format = "%Y-%h-%d-%H:%M:%S" # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
        f.write(timestamp + " : " + message + "\n")

#### **Task 2: Extraction of data**
- Write the function extract() to retrieve the information of the table to a Pandas DataFrame.

In [69]:
def extract(url):
    df = pd.read_excel(url)
    return df

- Write the function info_data() to get information about dataframe.

In [70]:
def info_data(df):
    print(df.head())
    print("Number of rows and columns are: ", df.shape)
    print()
    print("Information about dataframe: ")
    print(df.info())
    print("Number of unique atribute: ")
    print(df.nunique())
    print("Number of duplicate rows:" ,df.duplicated().sum())
    print()
    print("Number of missing values:" )
    print(df.isnull().sum())

#### **Task 3: Transformation of data**
- Change format type of dataframe, rename column of dataframe, fill null value, replace value.

- Write the function transform_teams_df() of **Team Dataframe**

In [71]:
def transform_teams_df(df):
    df_new = df

    # Changes all data type object to string
    df_new = df_new.astype("string")

    # Rename column
    df_new.rename(columns={'Name':'TeamName', 'NOC':'Country'}, inplace = True)
    return df_new

- Write the function transform_coaches_df() of **Coaches Dataframe**

In [72]:
def transform_coaches_df(df):
    df_new = df

    # Fill null value in Event column
    df_new['Event'] = df_new['Event'].fillna(value='N/A')

    # Changes all data type object to string
    df_new = df_new.astype("string")

    # Remame columns
    df_new.rename(columns={'NOC':'Country'}, inplace = True)
    return df_new

- Write the function transform_athletes_df() of **Athletes Dataframe**

In [73]:
def transform_athletes_df(df):
    df_new = df

    # Changes all data type object to string
    df_new = df_new.astype("string")

    # Remame columns
    df_new.rename(columns={'NOC':'Country'}, inplace = True)
    return df_new

- Write the function transform_gender_df() of **Gender Dataframe**

In [74]:
def transform_gender_df(df):
    df_new = df

    # Changes all data type object to string
    df_new['Discipline'] = df_new['Discipline'].astype("string")
    return df_new

- Write the function transform_medals_df() of **Medals Dataframe**

In [75]:
def transform_medals_df(df):
    df_new = df

    # Changes all data type object to string
    df_new['Team/NOC'] = df_new['Team/NOC'].astype("string")

    # Remame columns
    df_new.rename(columns={'Team/NOC':'TeamCountry'}, inplace = True)
    return df_new

- **Transform Dataframe for sql server.**

In [76]:
def transform_df_for_sql_db(TeamsDf: pd.DataFrame, column_name: str):
    def fix_column_name(column_value):
        index = column_value.find("'")
        if index > 0:
            return column_value[:index] + "'" + column_value[index:]
        return column_value

    df = TeamsDf.copy()
    df[column_name] = df[column_name].apply(fix_column_name)
    return df

#### **Task 4: Loading to CSV**
- Load the transformed dataframe to an output CSV file.
- **NOTE**: All processed data csv file in data/processed folder.

In [77]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(output_path, index=False)

#### **Task 5: Loading to Database**
- Load the transformed dataframe to a SQL database server as a table.

In [78]:
def connection_sql_server(cnxn_str):
    try:
        cnxn = pyodbc.connect(cnxn_str)
        print ("Success connection")
        cursor = cnxn.cursor()
        return cnxn, cursor
    except Exception as e:
        print("Error connection: " + str(e))

def create_table_sql_server(cnxn, cursor, query):
    try:
        cursor.execute(query)
        print("Table created successfully")
        cnxn.commit()
    except Exception as e:
        print("Error while creating table: " + str(e))

def load_to_db(df, cnxn, cursor, query):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    try:
        count = 0
        for i, row in df.iterrows():
            count = i
            para = tuple(row)
            cursor.execute(query % para)
            cnxn.commit()
        print("Data inserted ", count, " rows successfully")
    except Exception as e:
        print("Error while inserting data table: " + str(e))

#### **Task 6: Function to Run queries on Database**
- Run queries on the database table.

In [79]:
def run_query(cnxn, query):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    try:
        df = pd.read_sql(query, cnxn)
        return df
    except Exception as e:
        print("Error while selecting data table" + str(e))

#### **Running ETL Process**
- **Declaring known values**

In [80]:
path_data_raw = "./data/raw/"
log_file = "./code_log.txt"
output_path = "./data/processed/"

log_progress("Preliminaries complete. Initiating ETL process")

- **Check path data raw file**

In [81]:
for dirname, _, filenames in os.walk(path_data_raw):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/raw/Athletes.xlsx
./data/raw/Coaches.xlsx
./data/raw/EntriesGender.xlsx
./data/raw/Medals.xlsx
./data/raw/Teams.xlsx


- **Call extract() function:**

TeamsDf

In [82]:
url_team_raw = "./data/raw/Teams.xlsx"

TeamsDf = extract(url_team_raw)
print(TeamsDf)

log_progress("TeamsDf Data extraction complete. Initiating Transformation process")

              Name      Discipline                         NOC  Event
0          Belgium  3x3 Basketball                     Belgium    Men
1            China  3x3 Basketball  People's Republic of China    Men
2            China  3x3 Basketball  People's Republic of China  Women
3           France  3x3 Basketball                      France  Women
4            Italy  3x3 Basketball                       Italy  Women
..             ...             ...                         ...    ...
738   South Africa      Water Polo                South Africa  Women
739          Spain      Water Polo                       Spain    Men
740          Spain      Water Polo                       Spain  Women
741  United States      Water Polo    United States of America    Men
742  United States      Water Polo    United States of America  Women

[743 rows x 4 columns]


CoachesDf

In [83]:
url_coach_raw = "./data/raw/Coaches.xlsx"

CoachesDf = extract(url_coach_raw)
print(CoachesDf)

log_progress("CoachesDf Data extraction complete. Initiating Transformation process")

                       Name                         NOC         Discipline  \
0           ABDELMAGID Wael                       Egypt           Football   
1                 ABE Junya                       Japan         Volleyball   
2             ABE Katsuhiko                       Japan         Basketball   
3              ADAMA Cherif               Côte d'Ivoire           Football   
4                AGEBA Yuya                       Japan         Volleyball   
..                      ...                         ...                ...   
389  ZAMORA PEDREIRA Javier                       Spain         Basketball   
390      ZAMPIERI Francesca               Liechtenstein  Artistic Swimming   
391          ZHANG Xiaohuan  People's Republic of China  Artistic Swimming   
392              ZIJP Simon                 Netherlands             Hockey   
393           ZONDI Nkuliso                South Africa             Hockey   

     Event  
0      NaN  
1      NaN  
2      NaN  
3      NaN 

AthletesDf

In [84]:
url_athlete_raw = "./data/raw/Athletes.xlsx"

AthletesDf = extract(url_athlete_raw)
print(AthletesDf)

log_progress("AthletesDf Data extraction complete. Initiating Transformation process")

                        Name      NOC           Discipline
0            AALERUD Katrine   Norway         Cycling Road
1                ABAD Nestor    Spain  Artistic Gymnastics
2          ABAGNALE Giovanni    Italy               Rowing
3             ABALDE Alberto    Spain           Basketball
4              ABALDE Tamara    Spain           Basketball
...                      ...      ...                  ...
11080  ZWICKER Martin Detlef  Germany               Hockey
11081      ZWOLINSKA Klaudia   Poland         Canoe Slalom
11082           ZYKOVA Yulia      ROC             Shooting
11083      ZYUZINA Ekaterina      ROC              Sailing
11084        ZYZANSKA Sylwia   Poland              Archery

[11085 rows x 3 columns]


GenderDf

In [85]:
url_gender_raw = "./data/raw/EntriesGender.xlsx"

GenderDf = extract(url_gender_raw)
print(GenderDf)

log_progress("GenderDf Data extraction complete. Initiating Transformation process")

               Discipline  Female  Male  Total
0          3x3 Basketball      32    32     64
1                 Archery      64    64    128
2     Artistic Gymnastics      98    98    196
3       Artistic Swimming     105     0    105
4               Athletics     969  1072   2041
5               Badminton      86    87    173
6       Baseball/Softball      90   144    234
7              Basketball     144   144    288
8        Beach Volleyball      48    48     96
9                  Boxing     102   187    289
10           Canoe Slalom      41    41     82
11           Canoe Sprint     123   126    249
12  Cycling BMX Freestyle      10     9     19
13     Cycling BMX Racing      24    24     48
14  Cycling Mountain Bike      38    38     76
15           Cycling Road      70   131    201
16          Cycling Track      90    99    189
17                 Diving      72    71    143
18             Equestrian      73   125    198
19                Fencing     107   108    215
20           

MedalsDf

In [86]:
url_medal_raw = "./data/raw/Medals.xlsx"

MedalsDf = extract(url_medal_raw)
print(MedalsDf)

log_progress("MedalsDf Data extraction complete. Initiating Transformation process")

    Rank                    Team/NOC  Gold  Silver  Bronze  Total  \
0      1    United States of America    39      41      33    113   
1      2  People's Republic of China    38      32      18     88   
2      3                       Japan    27      14      17     58   
3      4               Great Britain    22      21      22     65   
4      5                         ROC    20      28      23     71   
..   ...                         ...   ...     ...     ...    ...   
88    86                       Ghana     0       0       1      1   
89    86                     Grenada     0       0       1      1   
90    86                      Kuwait     0       0       1      1   
91    86         Republic of Moldova     0       0       1      1   
92    86        Syrian Arab Republic     0       0       1      1   

    Rank by Total  
0               1  
1               2  
2               5  
3               4  
4               3  
..            ...  
88             77  
89         

- **Information about Dataframe**

Teams dataframe

In [87]:
info_data(TeamsDf)

      Name      Discipline                         NOC  Event
0  Belgium  3x3 Basketball                     Belgium    Men
1    China  3x3 Basketball  People's Republic of China    Men
2    China  3x3 Basketball  People's Republic of China  Women
3   France  3x3 Basketball                      France  Women
4    Italy  3x3 Basketball                       Italy  Women
Number of rows and columns are:  (743, 4)

Information about dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743 entries, 0 to 742
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        743 non-null    object
 1   Discipline  743 non-null    object
 2   NOC         743 non-null    object
 3   Event       743 non-null    object
dtypes: object(4)
memory usage: 23.3+ KB
None
Number of unique atribute: 
Name          146
Discipline     20
NOC            84
Event          36
dtype: int64
Number of duplicate rows: 0

Number of missing value

Coaches dataframe

In [88]:
info_data(CoachesDf)

              Name            NOC  Discipline Event
0  ABDELMAGID Wael          Egypt    Football   NaN
1        ABE Junya          Japan  Volleyball   NaN
2    ABE Katsuhiko          Japan  Basketball   NaN
3     ADAMA Cherif  Côte d'Ivoire    Football   NaN
4       AGEBA Yuya          Japan  Volleyball   NaN
Number of rows and columns are:  (394, 4)

Information about dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        394 non-null    object
 1   NOC         394 non-null    object
 2   Discipline  394 non-null    object
 3   Event       249 non-null    object
dtypes: object(4)
memory usage: 12.4+ KB
None
Number of unique atribute: 
Name          381
NOC            61
Discipline      9
Event           6
dtype: int64
Number of duplicate rows: 1

Number of missing values:
Name            0
NOC             0
Discipline      0
Eve

Athletes dataframe

In [89]:
info_data(AthletesDf)

                Name     NOC           Discipline
0    AALERUD Katrine  Norway         Cycling Road
1        ABAD Nestor   Spain  Artistic Gymnastics
2  ABAGNALE Giovanni   Italy               Rowing
3     ABALDE Alberto   Spain           Basketball
4      ABALDE Tamara   Spain           Basketball
Number of rows and columns are:  (11085, 3)

Information about dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11085 entries, 0 to 11084
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        11085 non-null  object
 1   NOC         11085 non-null  object
 2   Discipline  11085 non-null  object
dtypes: object(3)
memory usage: 259.9+ KB
None
Number of unique atribute: 
Name          11062
NOC             206
Discipline       46
dtype: int64
Number of duplicate rows: 1

Number of missing values:
Name          0
NOC           0
Discipline    0
dtype: int64


Gender dataframe

In [90]:
info_data(GenderDf)

            Discipline  Female  Male  Total
0       3x3 Basketball      32    32     64
1              Archery      64    64    128
2  Artistic Gymnastics      98    98    196
3    Artistic Swimming     105     0    105
4            Athletics     969  1072   2041
Number of rows and columns are:  (46, 4)

Information about dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Discipline  46 non-null     object
 1   Female      46 non-null     int64 
 2   Male        46 non-null     int64 
 3   Total       46 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.6+ KB
None
Number of unique atribute: 
Discipline    46
Female        38
Male          41
Total         41
dtype: int64
Number of duplicate rows: 0

Number of missing values:
Discipline    0
Female        0
Male          0
Total         0
dtype: int64


Medals dataframe

In [91]:
info_data(MedalsDf)

   Rank                    Team/NOC  Gold  Silver  Bronze  Total  \
0     1    United States of America    39      41      33    113   
1     2  People's Republic of China    38      32      18     88   
2     3                       Japan    27      14      17     58   
3     4               Great Britain    22      21      22     65   
4     5                         ROC    20      28      23     71   

   Rank by Total  
0              1  
1              2  
2              5  
3              4  
4              3  
Number of rows and columns are:  (93, 7)

Information about dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           93 non-null     int64 
 1   Team/NOC       93 non-null     object
 2   Gold           93 non-null     int64 
 3   Silver         93 non-null     int64 
 4   Bronze         93 non-null     int64 
 5

- **Call transform() function**

TeamsDf

In [92]:
TeamsDf = transform_teams_df(TeamsDf)
print(TeamsDf)

log_progress("TeamsDf Data transformation complete. Initiating Loading process")

          TeamName      Discipline                     Country  Event
0          Belgium  3x3 Basketball                     Belgium    Men
1            China  3x3 Basketball  People's Republic of China    Men
2            China  3x3 Basketball  People's Republic of China  Women
3           France  3x3 Basketball                      France  Women
4            Italy  3x3 Basketball                       Italy  Women
..             ...             ...                         ...    ...
738   South Africa      Water Polo                South Africa  Women
739          Spain      Water Polo                       Spain    Men
740          Spain      Water Polo                       Spain  Women
741  United States      Water Polo    United States of America    Men
742  United States      Water Polo    United States of America  Women

[743 rows x 4 columns]


In [93]:
TeamsDf

,TeamName,Discipline,Country,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women
...,...,...,...,...
738,South Africa,Water Polo,South Africa,Women
739,Spain,Water Polo,Spain,Men
740,Spain,Water Polo,Spain,Women
741,United States,Water Polo,United States of America,Men


CoachesDf

In [94]:
CoachesDf = transform_coaches_df(CoachesDf)
print(CoachesDf)

log_progress("CoachesDf Data transformation complete. Initiating Loading process")

                       Name                     Country         Discipline  \
0           ABDELMAGID Wael                       Egypt           Football   
1                 ABE Junya                       Japan         Volleyball   
2             ABE Katsuhiko                       Japan         Basketball   
3              ADAMA Cherif               Côte d'Ivoire           Football   
4                AGEBA Yuya                       Japan         Volleyball   
..                      ...                         ...                ...   
389  ZAMORA PEDREIRA Javier                       Spain         Basketball   
390      ZAMPIERI Francesca               Liechtenstein  Artistic Swimming   
391          ZHANG Xiaohuan  People's Republic of China  Artistic Swimming   
392              ZIJP Simon                 Netherlands             Hockey   
393           ZONDI Nkuliso                South Africa             Hockey   

     Event  
0      N/A  
1      N/A  
2      N/A  
3      N/A 

In [95]:
CoachesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        394 non-null    string
 1   Country     394 non-null    string
 2   Discipline  394 non-null    string
 3   Event       394 non-null    string
dtypes: string(4)
memory usage: 12.4 KB


AthletesDf

In [96]:
AthletesDf = transform_athletes_df(AthletesDf)
print(AthletesDf)

log_progress("AthletesDf Data transformation complete. Initiating Loading process")

                        Name  Country           Discipline
0            AALERUD Katrine   Norway         Cycling Road
1                ABAD Nestor    Spain  Artistic Gymnastics
2          ABAGNALE Giovanni    Italy               Rowing
3             ABALDE Alberto    Spain           Basketball
4              ABALDE Tamara    Spain           Basketball
...                      ...      ...                  ...
11080  ZWICKER Martin Detlef  Germany               Hockey
11081      ZWOLINSKA Klaudia   Poland         Canoe Slalom
11082           ZYKOVA Yulia      ROC             Shooting
11083      ZYUZINA Ekaterina      ROC              Sailing
11084        ZYZANSKA Sylwia   Poland              Archery

[11085 rows x 3 columns]


In [97]:
AthletesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11085 entries, 0 to 11084
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        11085 non-null  string
 1   Country     11085 non-null  string
 2   Discipline  11085 non-null  string
dtypes: string(3)
memory usage: 259.9 KB


GenderDf

In [98]:
GenderDf = transform_gender_df(GenderDf)
print(GenderDf)

log_progress("GenderDf Data transformation complete. Initiating Loading process")

               Discipline  Female  Male  Total
0          3x3 Basketball      32    32     64
1                 Archery      64    64    128
2     Artistic Gymnastics      98    98    196
3       Artistic Swimming     105     0    105
4               Athletics     969  1072   2041
5               Badminton      86    87    173
6       Baseball/Softball      90   144    234
7              Basketball     144   144    288
8        Beach Volleyball      48    48     96
9                  Boxing     102   187    289
10           Canoe Slalom      41    41     82
11           Canoe Sprint     123   126    249
12  Cycling BMX Freestyle      10     9     19
13     Cycling BMX Racing      24    24     48
14  Cycling Mountain Bike      38    38     76
15           Cycling Road      70   131    201
16          Cycling Track      90    99    189
17                 Diving      72    71    143
18             Equestrian      73   125    198
19                Fencing     107   108    215
20           

In [99]:
GenderDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Discipline  46 non-null     string
 1   Female      46 non-null     int64 
 2   Male        46 non-null     int64 
 3   Total       46 non-null     int64 
dtypes: int64(3), string(1)
memory usage: 1.6 KB


MedalsDf

In [100]:
MedalsDf = transform_medals_df(MedalsDf)
print(MedalsDf)

log_progress("MedalsDf Data transformation complete. Initiating Loading process")

    Rank                 TeamCountry  Gold  Silver  Bronze  Total  \
0      1    United States of America    39      41      33    113   
1      2  People's Republic of China    38      32      18     88   
2      3                       Japan    27      14      17     58   
3      4               Great Britain    22      21      22     65   
4      5                         ROC    20      28      23     71   
..   ...                         ...   ...     ...     ...    ...   
88    86                       Ghana     0       0       1      1   
89    86                     Grenada     0       0       1      1   
90    86                      Kuwait     0       0       1      1   
91    86         Republic of Moldova     0       0       1      1   
92    86        Syrian Arab Republic     0       0       1      1   

    Rank by Total  
0               1  
1               2  
2               5  
3               4  
4               3  
..            ...  
88             77  
89         

In [101]:
MedalsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           93 non-null     int64 
 1   TeamCountry    93 non-null     string
 2   Gold           93 non-null     int64 
 3   Silver         93 non-null     int64 
 4   Bronze         93 non-null     int64 
 5   Total          93 non-null     int64 
 6   Rank by Total  93 non-null     int64 
dtypes: int64(6), string(1)
memory usage: 5.2 KB


- **Call load_to_csv()**

TeamsDf

In [102]:
teamDf_output_path = output_path + "Teams.csv"

load_to_csv(TeamsDf, teamDf_output_path)

log_progress("TeamsDf Data saved to CSV file")

CoachesDf

In [103]:
coachDf_output_path = output_path + "Coaches.csv"

load_to_csv(CoachesDf, coachDf_output_path)

log_progress("CoachesDf Data saved to CSV file")

AthletesDf

In [104]:
athleteDf_output_path = output_path + "Athletes.csv"

load_to_csv(AthletesDf, athleteDf_output_path)

log_progress("AthletesDf Data saved to CSV file")

GenderDf

In [105]:
genderDf_output_path = output_path + "Gender.csv"

load_to_csv(GenderDf, genderDf_output_path)

log_progress("GenderDf Data saved to CSV file")

MedalsDf

In [106]:
medalDf_output_path = output_path + "Medals.csv"

load_to_csv(MedalsDf, medalDf_output_path)

log_progress("MedalDf Data saved to CSV file")

- **Initiate MS SQL SERVER connection**

In [127]:
server = "localhost\SQLEXPRESS"
database = "OlympicTokyo"
cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
                      "Server="+ server +";"
                      "Database="+ database +";"
                      "Trusted_Connection=yes;")

In [129]:
cnxn, cursor = connection_sql_server(cnxn_str)

Success connection


Create table

In [130]:
query = "CREATE TABLE teams(TeamName VARCHAR(50), Discipline VARCHAR(50), Country VARCHAR(50), Event VARCHAR(50))"
create_table_sql_server(cnxn, cursor, query)

Table created successfully


In [131]:
query = "CREATE TABLE coaches(Name VARCHAR(50), Country VARCHAR(50), Discipline VARCHAR(50), Event VARCHAR(50))"
create_table_sql_server(cnxn, cursor, query)

Table created successfully


In [132]:
query = "CREATE TABLE athletes(Name VARCHAR(50), Country VARCHAR(50), Discipline VARCHAR(50))"
create_table_sql_server(cnxn, cursor, query)

Table created successfully


In [133]:
query = "CREATE TABLE gender(Discipline VARCHAR(50), Female INT, Male INT, Total INT)"
create_table_sql_server(cnxn, cursor, query)

Table created successfully


In [134]:
query = "CREATE TABLE medals(Rank INT, TeamCountry VARCHAR(50), Gold INT, Silver INT, Bronze INT, Total INT, RankByTotal INT)"
create_table_sql_server(cnxn, cursor, query)

Table created successfully


- **Transform data column for SQL SERVER**

In [135]:
TeamsDf = transform_df_for_sql_db(TeamsDf, 'Country')
TeamsDf = transform_df_for_sql_db(TeamsDf, 'Event')
print(TeamsDf)

log_progress("TeamsDf Data transformation for SQL Server complete. Initiating Loading process")

          TeamName      Discipline                      Country  Event
0          Belgium  3x3 Basketball                      Belgium    Men
1            China  3x3 Basketball  People''s Republic of China    Men
2            China  3x3 Basketball  People''s Republic of China  Women
3           France  3x3 Basketball                       France  Women
4            Italy  3x3 Basketball                        Italy  Women
..             ...             ...                          ...    ...
738   South Africa      Water Polo                 South Africa  Women
739          Spain      Water Polo                        Spain    Men
740          Spain      Water Polo                        Spain  Women
741  United States      Water Polo     United States of America    Men
742  United States      Water Polo     United States of America  Women

[743 rows x 4 columns]


In [136]:
CoachesDf = transform_df_for_sql_db(CoachesDf, 'Country')

log_progress("CoachesDf Data transformation for SQL Server complete. Initiating Loading process")

In [137]:
AthletesDf = transform_df_for_sql_db(AthletesDf, 'Name')
AthletesDf = transform_df_for_sql_db(AthletesDf, 'Country')
print(AthletesDf)

log_progress("AthletesDf Data transformation for SQL Server complete. Initiating Loading process")

                        Name  Country           Discipline
0            AALERUD Katrine   Norway         Cycling Road
1                ABAD Nestor    Spain  Artistic Gymnastics
2          ABAGNALE Giovanni    Italy               Rowing
3             ABALDE Alberto    Spain           Basketball
4              ABALDE Tamara    Spain           Basketball
...                      ...      ...                  ...
11080  ZWICKER Martin Detlef  Germany               Hockey
11081      ZWOLINSKA Klaudia   Poland         Canoe Slalom
11082           ZYKOVA Yulia      ROC             Shooting
11083      ZYUZINA Ekaterina      ROC              Sailing
11084        ZYZANSKA Sylwia   Poland              Archery

[11085 rows x 3 columns]


In [138]:
MedalsDf = transform_df_for_sql_db(MedalsDf, 'TeamCountry')
print(MedalsDf)

log_progress("MedalsDf Data transformation for SQL Server complete. Initiating Loading process")

    Rank                  TeamCountry  Gold  Silver  Bronze  Total  \
0      1     United States of America    39      41      33    113   
1      2  People''s Republic of China    38      32      18     88   
2      3                        Japan    27      14      17     58   
3      4                Great Britain    22      21      22     65   
4      5                          ROC    20      28      23     71   
..   ...                          ...   ...     ...     ...    ...   
88    86                        Ghana     0       0       1      1   
89    86                      Grenada     0       0       1      1   
90    86                       Kuwait     0       0       1      1   
91    86          Republic of Moldova     0       0       1      1   
92    86         Syrian Arab Republic     0       0       1      1   

    Rank by Total  
0               1  
1               2  
2               5  
3               4  
4               3  
..            ...  
88             77  

- **Call load_to_db()**

In [139]:
query = "INSERT INTO teams VALUES('%s', '%s', '%s', '%s')"
load_to_db(TeamsDf, cnxn, cursor, query)

Data inserted  742  rows successfully


In [140]:
query = "INSERT INTO coaches VALUES('%s', '%s', '%s', '%s')"
load_to_db(CoachesDf, cnxn, cursor, query)

Data inserted  393  rows successfully


In [141]:
query = "INSERT INTO athletes VALUES('%s', '%s', '%s')"
load_to_db(AthletesDf, cnxn, cursor, query)

Data inserted  11084  rows successfully


In [142]:
query = "INSERT INTO gender VALUES('%s', '%g', '%g', '%g')"
load_to_db(GenderDf, cnxn, cursor, query)

Data inserted  45  rows successfully


In [143]:
query = "INSERT INTO medals VALUES('%g', '%s', '%g', '%g', '%g', '%g', '%g')"
load_to_db(MedalsDf, cnxn, cursor, query)

Data inserted  92  rows successfully


- **Call run_query()**

Number of participants in each country

In [109]:
query = """SELECT TOP 20
	Country
	, COUNT(*) AS NumOfAthletes
    FROM athletes
    GROUP BY Country
    ORDER BY COUNT(*) DESC"""

df_top20_num_athletes_of_country = run_query(cnxn, query)
print(df_top20_num_athletes_of_country)

                       Country  NumOfAthletes
0     United States of America            615
1                        Japan            586
2                    Australia            470
3   People's Republic of China            401
4                      Germany            400
5                       France            377
6                       Canada            368
7                Great Britain            366
8                        Italy            356
9                        Spain            324
10                         ROC            318
11                      Brazil            291
12                 Netherlands            274
13           Republic of Korea            223
14                 New Zealand            202
15                      Poland            195
16                   Argentina            180
17                South Africa            171
18                      Mexico            155
19                     Hungary            155


The most Discipline in the olympiade

In [110]:
query = """SELECT
	Discipline
	, COUNT(*) AS NumOfAthletes
    FROM athletes
    GROUP BY Discipline
    ORDER BY COUNT(*) DESC"""

df_num_athletes_of_discipline = run_query(cnxn, query)
print(df_num_athletes_of_discipline)

               Discipline  NumOfAthletes
0               Athletics           2068
1                Swimming            743
2                Football            567
3                  Rowing            496
4                  Hockey            406
5                    Judo            373
6                Handball            343
7                Shooting            342
8                 Sailing            336
9            Rugby Sevens            283
10             Basketball            280
11              Wrestling            279
12             Volleyball            274
13                 Boxing            270
14             Water Polo            269
15                Fencing            249
16             Equestrian            237
17           Canoe Sprint            236
18      Baseball/Softball            220
19          Cycling Track            208
20           Cycling Road            190
21          Weightlifting            187
22    Artistic Gymnastics            187
23              

Top 20 countries in terms of number of Coaches

In [111]:
query = """SELECT TOP 20
	Country
	, COUNT(*) AS NumOfCoach
    FROM coaches
    GROUP BY Country
    ORDER BY COUNT(*) DESC"""

df_num_coach_of_country = run_query(cnxn, query)
print(df_num_coach_of_country)

                       Country  NumOfCoach
0                        Japan          35
1                        Spain          28
2     United States of America          28
3                    Australia          22
4                       Canada          16
5                        Italy          14
6                        Egypt          12
7   People's Republic of China          12
8                          ROC          12
9                 South Africa          12
10                   Argentina          11
11                      France          10
12                 Netherlands          10
13                   Venezuela          10
14                     Germany           9
15                     Nigeria           9
16                      Mexico           8
17                 New Zealand           8
18                      Brazil           7
19               Great Britain           7


- **Close MS SQL SERVER connection**

In [144]:
#Close connection
cursor.close()
cnxn.close()

#### **Task 7: Verify log entries**
- Upon successful completion of execution, you should see all the relevant entries made in the log file in relation to the stages of code execution.

In [145]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)

2024-Nov-27-14:10:23 : Preliminaries complete. Initiating ETL process
2024-Nov-27-14:10:24 : TeamsDf Data extraction complete. Initiating Transformation process
2024-Nov-27-14:10:24 : CoachesDf Data extraction complete. Initiating Transformation process
2024-Nov-27-14:10:25 : AthletesDf Data extraction complete. Initiating Transformation process
2024-Nov-27-14:10:25 : GenderDf Data extraction complete. Initiating Transformation process
2024-Nov-27-14:10:25 : MedalsDf Data extraction complete. Initiating Transformation process
2024-Nov-27-14:11:10 : TeamsDf Data transformation complete. Initiating Loading process
2024-Nov-27-14:11:16 : CoachesDf Data transformation complete. Initiating Loading process
2024-Nov-27-14:11:24 : AthletesDf Data transformation complete. Initiating Loading process
2024-Nov-27-14:11:29 : GenderDf Data transformation complete. Initiating Loading process
2024-Nov-27-14:11:34 : MedalsDf Data transformation complete. Initiating Loading process
2024-Nov-27-14:11:43 

# Data Analysis

In [114]:
import plotly.express as px
import matplotlib.pyplot as plt

### Number of participants in each country

Using dataframe get data

In [115]:
data = AthletesDf.Country.value_counts()
fig=px.bar(data[:20], title="Top 20 countries in terms of number of participants :")
fig.update_xaxes(title_text="Countries")

Get data from sql server by SQL query

In [116]:
#Using 
data = df_top20_num_athletes_of_country
fig = px.bar(data, title = "Top 20 countries in terms of number of participants :", x = data['Country'], y = data['NumOfAthletes'])
fig.update_xaxes(title_text="Countries")

![alt](./image/NumOfParticipantsInEachCountryPlot.png)

Note :
According to this graph, we can observe that the USA has the highest number of participants, followed by Japan and Australia but Mexico and Hungary have the least number of participants.

### Number of Discipline in the olympiade

Using dataframe get data

In [117]:
data = AthletesDf.Discipline.value_counts()
num_disc = data.values
num_disc_index = data.index
plt.figure(figsize = (12,7))
fig= px.bar(data , x=num_disc_index, y=num_disc ,title="The most Discipline in the olympiade",labels={"x":"Athletes","y":"Number"})
fig.update_xaxes(title_text="Discipline")
fig.show()

<Figure size 1200x700 with 0 Axes>

Get data from sql server by SQL query

In [118]:
data = df_num_athletes_of_discipline
plt.figure(figsize = (12,7))
fig= px.bar(data , x=data['Discipline'], y=['NumOfAthletes'],title="The number of Athletes by Discipline in the olympic")
fig.update_xaxes(title_text="Discipline")
fig.show()

<Figure size 1200x700 with 0 Axes>

![alt](./image/NumOfDisciplineInTheOlympiadePlot.png)

Note: 
Athletics is the most popular discipline in the olympics; it has most number of participants. Swimming and Football are the next most popular ones but Cycling BMX Freestyle is the least popular discipline with only 19 participants

### Number of Coaches in each country

Using dataframe get data

In [119]:
data2 = CoachesDf.Country.value_counts()
fig = px.bar(data2[:20], title="Top 20 countries in terms of number of Coaches :")
fig.update_xaxes(title_text="Countries")

Get data from sql server by SQL query

In [120]:
data = df_num_coach_of_country
fig = px.bar(data, title="Top 20 countries in terms of number of Coaches :", x = data['Country'], y = data['NumOfCoach'])
fig.update_xaxes(title_text="Countries")

![alt](./image/NumberOfCoachesPlot.png)

Note: 
Japan contributes to maximum number of coaches followed by USA and Spain but Great Britain and Republic of Korea have the least number of coaches.

### Number of Medals in each country

In [121]:
px.bar(MedalsDf, x="TeamCountry", y="Total", color="Gold", title="Top Countries in terms of number of medals: ")

![alt](./image/NumberOfMedalsPlot.png)

Note: 
USA is on the top follwed by China and Japan. We note that Germany and Australia are in top 5 countries participants are coming from but not in top 5 countries having maximum medals.Whereas USA, China, Japan holds top position in both number of participants and medals won.

### Top Gold Medals by each country

In [122]:
px.bar(MedalsDf, x="TeamCountry", y="Gold",title="Gold Medals")

![alt](./image/GoldMedelsPlot.png)

### Top Silver Medals by each country

In [123]:
MedalsSortBySilverDf = MedalsDf.sort_values(by='Silver', ascending=False)
px.bar(MedalsSortBySilverDf, x="TeamCountry", y="Silver" ,title="Silver Medals")

![alt](./image/SilverMedalsPlot.png)

### Top Bronze Medals by each country

In [124]:
MedalsSortByBronzeDf = MedalsDf.sort_values(by='Bronze', ascending=False)
px.bar(MedalsSortByBronzeDf, x="TeamCountry", y="Bronze" ,title="Bronze Medals")

![alt](./image/BronzeMedalsPlot.png)

Note: 
Note that the USA has the maximum number of medals in Gold, Silver, and Bronze.

### Number of participants in each Discipline

In [125]:
ParticipantsSortByTotalDf = GenderDf.sort_values(by='Total', ascending=False)
px.bar(ParticipantsSortByTotalDf, x="Discipline", y="Total", color="Discipline", title="Total participants in each Discipline : ")

![alt](./image/TotalParticipantsInEachDisciplinePlot.png)

Note:
We conclude that Atheletics has maximum number of participants , Swimming has second highest number of participants and Cycling has minimum number of participants.

### Number of Male and Female in each Discipline

In [126]:
TotalGenderSortByTotal = GenderDf.sort_values(by='Total', ascending=False)
px.bar(TotalGenderSortByTotal, x="Discipline", y=["Male", "Female"], barmode="group")

![alt](./image/NumberOfMaleAndFemaleInEachDisciplinePlot.png)

Note:
Females participate in all the disciplines but comparatively less than men.

# Final Analysis
- The majority of athletes who participated in the 2021 (2020) Tokyo Olympics originated from the countries USA, Japan, and Australia. This can be attributed to factors such as the strong sporting culture, high levels of investment in sports infrastructure, and extensive talent development programs in these nations.

- Athletics is the most popular discipline in the Olympics. This could be due to its broad appeal as a fundamental and accessible sport that embodies the essence of the Games' spirit, attracting athletes and fans from diverse backgrounds.

- Japan produces the most coaches and US after them. Again this could be because of the vast culture of sports in these nations and as athletes grow preparing from a much younger age, they gain a lot of experience, resultingly becoming coaches.

- The USA has garnered the highest number of medals in Gold, Silver, and Bronze. This achievement can be attributed to the country's significant investment in sports infrastructure, robust training programs, and the extensive support provided to its athletes.